In [1]:
import torch
import pandas as pd
import numpy as np
import os

import random
from copy import deepcopy
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
sensors = ['Cz', 'Fz', 'Fp1', 'F7', 'F3', 'FC1', 'C3', 'FC5', 'FT9', 'T7', 'CP5', 'CP1',
          'P3', 'P7', 'PO9', 'O1', 'Pz', 'Oz', 'O2', 'PO10', 'P8', 'P4', 'CP2', 'CP6',
          'T8', 'FT10', 'FC6', 'C4', 'FC2', 'F4', 'F8', 'Fp2']

EEG_cols = ['EEG.' + s for s in sensors]
CQ_cols = ['CQ.' + s for s in sensors]
EQ_cols = ['EQ.' + s for s in sensors]

In [3]:
def divide_df(df):
    # df = pd.read_csv(path, header=1)
    
    EEG_df = deepcopy(df[EEG_cols])
    CQ_df = deepcopy(df[CQ_cols])
    EQ_df = deepcopy(df[EQ_cols])
    
    for i in range(len(EQ_df)):
        if not np.isnan(EQ_df.iloc[i]['EQ.Cz']):
            start_idx = i
            break
        
    EEG_df = EEG_df[start_idx:].reset_index().drop(columns=['index'])
    CQ_df = CQ_df[start_idx:].reset_index().drop(columns=['index'])
    EQ_df = EQ_df[start_idx:].reset_index().drop(columns=['index'])

    EQ_df = EQ_df.fillna(method='ffill')
    
    return EEG_df, CQ_df, EQ_df

def remove_CQ_is_zero(EEG_df, CQ_df, EQ_df):
    print('Before:', len(EEG_df), len(CQ_df), len(EQ_df))
    
    remove_indexs = []
    for i in range(len(CQ_df)):
        if 0 in list(CQ_df.iloc[i]):
            remove_indexs.append(i)
    print('Remove Num:', len(remove_indexs))
    
    EEG_df = EEG_df.drop(remove_indexs, axis=0)
    EQ_df = EQ_df.drop(remove_indexs, axis=0)
    
    print('After:', len(EEG_df), len(CQ_df), len(EQ_df))
    
    return EEG_df, EQ_df

def weight_and_scale(EEG_df, EQ_df):
    EQ_df.columns = EEG_df.columns
    
    EQ_df = EQ_df.replace(0, 0.5)
    EQ_df = EQ_df.replace(1, 1.0)
    EQ_df = EQ_df.replace(2, 1.5)
    EQ_df = EQ_df.replace(3, 2.0)
    EQ_df = EQ_df.replace(4, 2.5)
    
    weight_EEG_df = EEG_df.mul(EQ_df)
    
    scaler = MinMaxScaler()
    output = scaler.fit_transform(weight_EEG_df.values)
    
    weight_EEG_df_scale = pd.DataFrame(output, columns = EEG_df.columns)

    scaler = MinMaxScaler()
    output = scaler.fit_transform(EEG_df.values)
    no_weight_EEG_scale = pd.DataFrame(output, columns = EEG_df.columns)
    
    return weight_EEG_df_scale, no_weight_EEG_scale

In [4]:
def save_preprocessing_data(task):
    
    error_column_list = ['정찬영.csv', '유상봉.csv', '서성범.csv', '박정수.csv', '최석환.csv', '이재훈.csv','윤찬영.csv']
    folder_path = './ori/' + task
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    dataframes = []  # CSV 파일을 저장할 빈 리스트
    
    for csv_file in csv_files:
        print(csv_file, folder_path)
        if csv_file in error_column_list:
            df = pd.read_csv(folder_path + '/' + csv_file ,header=1)
        else:
            df = pd.read_csv(folder_path + '/' + csv_file)
        
        if len(df) != 0:
            print('preprocessing start')
            EEG_df, CQ_df, EQ_df = divide_df(df)
            EEG_df, EQ_df = remove_CQ_is_zero(EEG_df, CQ_df, EQ_df)

            weight_EEG_df_scale, no_weight_EEG_scale = weight_and_scale(EEG_df, EQ_df)

            weight_EEG_df_scale.to_csv('./preprocessing/weight/' + task + '/' + csv_file, index=False)
            no_weight_EEG_scale.to_csv('./preprocessing/noweight/' + task + '/' + csv_file, index=False)


In [39]:
save_preprocessing_data('1back')

권하윤.csv ./1back
preprocessing start
Before: 23443 23443 23443
Remove Num: 0
After: 23443 23443 23443
김민채.csv ./1back
동기창.csv ./1back
preprocessing start
Before: 23416 23416 23416
Remove Num: 0
After: 23416 23416 23416
류국봉.csv ./1back
박상욱.csv ./1back
preprocessing start
Before: 23411 23411 23411
Remove Num: 0
After: 23411 23411 23411
박정수.csv ./1back
preprocessing start
Before: 79855 79855 79855
Remove Num: 717
After: 79138 79855 79138
박지원.csv ./1back
preprocessing start
Before: 23514 23514 23514
Remove Num: 0
After: 23514 23514 23514
서성범.csv ./1back
preprocessing start
Before: 78834 78834 78834
Remove Num: 4469
After: 74365 78834 74365
손건영.csv ./1back
손재호.csv ./1back
preprocessing start
Before: 23431 23431 23431
Remove Num: 0
After: 23431 23431 23431
유민.csv ./1back
preprocessing start
Before: 23418 23418 23418
Remove Num: 0
After: 23418 23418 23418
유상봉.csv ./1back
preprocessing start
Before: 78995 78995 78995
Remove Num: 0
After: 78995 78995 78995
윤찬영.csv ./1back
preprocessing start
Bef

In [40]:
save_preprocessing_data('3back')

권하윤.csv ./3back
preprocessing start
Before: 23459 23459 23459
Remove Num: 0
After: 23459 23459 23459
김민채.csv ./3back
preprocessing start
Before: 23491 23491 23491
Remove Num: 0
After: 23491 23491 23491
동기창.csv ./3back
preprocessing start
Before: 23440 23440 23440
Remove Num: 0
After: 23440 23440 23440
류국봉.csv ./3back
박상욱.csv ./3back
preprocessing start
Before: 23500 23500 23500
Remove Num: 0
After: 23500 23500 23500
박정수.csv ./3back
preprocessing start
Before: 79960 79960 79960
Remove Num: 0
After: 79960 79960 79960
박지원.csv ./3back
서성범.csv ./3back
preprocessing start
Before: 78998 78998 78998
Remove Num: 0
After: 78998 78998 78998
손건영.csv ./3back
손재호.csv ./3back
preprocessing start
Before: 23440 23440 23440
Remove Num: 0
After: 23440 23440 23440
유민.csv ./3back
preprocessing start
Before: 24168 24168 24168
Remove Num: 0
After: 24168 24168 24168
유상봉.csv ./3back
preprocessing start
Before: 78979 78979 78979
Remove Num: 0
After: 78979 78979 78979
윤찬영.csv ./3back
preprocessing start
Before: 

In [42]:
save_preprocessing_data('10')

박정수.csv ./10
preprocessing start
Before: 79905 79905 79905
Remove Num: 0
After: 79905 79905 79905
서성범.csv ./10
preprocessing start
Before: 79046 79046 79046
Remove Num: 191
After: 78855 79046 78855
유상봉.csv ./10
preprocessing start
Before: 79435 79435 79435
Remove Num: 0
After: 79435 79435 79435
윤찬영.csv ./10
preprocessing start
Before: 80361 80361 80361
Remove Num: 0
After: 80361 80361 80361
이재훈.csv ./10
preprocessing start
Before: 77782 77782 77782
Remove Num: 6331
After: 71451 77782 71451
정찬영.csv ./10
preprocessing start
Before: 38819 38819 38819
Remove Num: 0
After: 38819 38819 38819
최석환.csv ./10
preprocessing start
Before: 79003 79003 79003
Remove Num: 0
After: 79003 79003 79003


In [43]:
save_preprocessing_data('20')

박정수.csv ./20
preprocessing start
Before: 78536 78536 78536
Remove Num: 189
After: 78347 78536 78347
서성범.csv ./20
preprocessing start
Before: 78968 78968 78968
Remove Num: 0
After: 78968 78968 78968
유상봉.csv ./20
preprocessing start
Before: 79101 79101 79101
Remove Num: 0
After: 79101 79101 79101
윤찬영.csv ./20
preprocessing start
Before: 80435 80435 80435
Remove Num: 64
After: 80371 80435 80371
이재훈.csv ./20
preprocessing start
Before: 79070 79070 79070
Remove Num: 717
After: 78353 79070 78353
정찬영.csv ./20
preprocessing start
Before: 78901 78901 78901
Remove Num: 0
After: 78901 78901 78901
최석환.csv ./20
preprocessing start
Before: 79274 79274 79274
Remove Num: 0
After: 79274 79274 79274


# 여기아래는 실행 안시켜도됨

In [44]:
def weight(EEG_df, EQ_df):
    EQ_df.columns = EEG_df.columns
    
    EQ_df = EQ_df.replace(0, 0.5)
    EQ_df = EQ_df.replace(1, 1.0)
    EQ_df = EQ_df.replace(2, 1.5)
    EQ_df = EQ_df.replace(3, 2.0)
    EQ_df = EQ_df.replace(4, 2.5)
    
    weight_EEG_df = EEG_df.mul(EQ_df)
    
    return weight_EEG_df, EEG_df

def save_preprocessing_data_non_scale(task):
    
    error_column_list = ['정찬영.csv', '유상봉.csv', '서성범.csv', '박정수.csv', '최석환.csv', '이재훈.csv','윤찬영.csv']
    folder_path = './ori/' + task
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    dataframes = []  # CSV 파일을 저장할 빈 리스트
    
    for csv_file in csv_files:
        print(csv_file, folder_path)
        if csv_file in error_column_list:
            df = pd.read_csv(folder_path + '/' + csv_file ,header=1)
        else:
            df = pd.read_csv(folder_path + '/' + csv_file)
        
        if len(df) != 0:
            print('preprocessing start')
            EEG_df, CQ_df, EQ_df = divide_df(df)
            EEG_df, EQ_df = remove_CQ_is_zero(EEG_df, CQ_df, EQ_df)

            weight_EEG_df, EEG_df = weight(EEG_df, EQ_df)

            weight_EEG_df.to_csv('./preprocessing/nonscale/weight/' + task + '/' + csv_file, index=False)
            EEG_df.to_csv('./preprocessing/nonscale/noweight/' + task + '/' + csv_file, index=False)


In [47]:
save_preprocessing_data_non_scale('3back')
save_preprocessing_data_non_scale('1back')
save_preprocessing_data_non_scale('10')
save_preprocessing_data_non_scale('20')

권하윤.csv ./3back
preprocessing start
Before: 23459 23459 23459
Remove Num: 0
After: 23459 23459 23459
김민채.csv ./3back
preprocessing start
Before: 23491 23491 23491
Remove Num: 0
After: 23491 23491 23491
동기창.csv ./3back
preprocessing start
Before: 23440 23440 23440
Remove Num: 0
After: 23440 23440 23440
류국봉.csv ./3back
박상욱.csv ./3back
preprocessing start
Before: 23500 23500 23500
Remove Num: 0
After: 23500 23500 23500
박정수.csv ./3back
preprocessing start
Before: 79960 79960 79960
Remove Num: 0
After: 79960 79960 79960
박지원.csv ./3back
서성범.csv ./3back
preprocessing start
Before: 78998 78998 78998
Remove Num: 0
After: 78998 78998 78998
손건영.csv ./3back
손재호.csv ./3back
preprocessing start
Before: 23440 23440 23440
Remove Num: 0
After: 23440 23440 23440
유민.csv ./3back
preprocessing start
Before: 24168 24168 24168
Remove Num: 0
After: 24168 24168 24168
유상봉.csv ./3back
preprocessing start
Before: 78979 78979 78979
Remove Num: 0
After: 78979 78979 78979
윤찬영.csv ./3back
preprocessing start
Before: 